In [69]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs
import os
from string import Template

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
tbs = df_sp["TableName"].unique()

# print(tbs)

def fmtViewColumns(r):
    split, col, colDscr, dataType, isOrderBy = "\n\t\t", r["Column"], r["ColDscr"], r["Type"], r["orderBy"]
    dataType = "right" if (dataType in ["int","float","decical"]) else "left"
    tmp = Template("{\n\t\tname: '$col',\n\t\tlabel: '$colDscr',\n\t\talign: '$dataType',\n\t\tfield: '$col',\n\t\tsortable: true,\n},\n")
    return tmp.substitute(split = split, col = col, colDscr= colDscr, dataType = dataType) if(isOrderBy =="V") else ""
    
def fmcViewFilterItem(r):
    split, col, colDscr, filterItem = "\n\t\t", r["Column"], r["ColDscr"], r["filterItem"]
    tmp = Template("{$split label: '$colDscr',$split value: '$col',$split placeholder: '請輸入$colDscr',\n},\n")
    return tmp.substitute(split = split ,col = col, colDscr= colDscr) if(filterItem =="V") else ""


def genJSCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]

    target = dfc[dfc["orderBy"] == "V"]
    target = target[target["Column"]!="updateDT"]

    pt_orderByCols = "".join(target["orderBy2"].tolist())[:-1]
    pt_filterItem = "".join(target["filterItem2"].tolist())[:-1]

    tmpJSFile = codecs.open(f"template/JS/views/admin/index.vue",mode="r",encoding="utf-8", buffering=-1).read()
    t = Template(tmpJSFile).substitute(pt_TableName = pt_TableName, pt_tableName = pt_tableName, pt_table_name = pt_table_name, pt_orderByCols = pt_orderByCols, pt_filterItem = pt_filterItem)
    
    # print(tmpJSFile)
    fileName = f"JS/src/views/admin/{pt_table_name}/index.vue"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()


for tb in tbs:

    # print(tb)
    if tb=="PcPhone":
        dfm = df_sp[df_sp["TableName"] == tb]
        dfc = df[df["TableName"] == tb]
        dfc["orderBy2"] = dfc.apply(fmtViewColumns, axis=1)
        dfc["filterItem2"] = dfc.apply(fmcViewFilterItem, axis=1)
        print(dfc)
        genJSCode(tb, dfm, dfc)
    


    Schema TableName PK        Column  ColDscr       Type maxLen isNull  \
51  agdSet   PcPhone  V         seqNo  Seq No.        int                 
52  agdSet   PcPhone     computerName     電腦名稱   nvarchar     50          
53  agdSet   PcPhone       computerIP    IP 位址   nvarchar     50          
54  agdSet   PcPhone          extCode     分機號碼   nvarchar     20          
55  agdSet   PcPhone             memo       備註   nvarchar    600          
56  agdSet   PcPhone          creator      建立者    varchar     20          
57  agdSet   PcPhone          updator      異動者    varchar     20          
58  agdSet   PcPhone         createDT     建立時間  datetime2      7          
59  agdSet   PcPhone         updateDT     異動時間  datetime2      7          

          autoGen NonCluster isExist httpGet httpUpdate httpInsert queryWhere  \
51  IDENTITY(1,1)                  V       1          1                         
52                                               CP0001     CP0001     CP0001   
53    

C:\Users\user\AppData\Local\Temp/ipykernel_5796/1826565885.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["orderBy2"] = dfc.apply(fmtViewColumns, axis=1)
C:\Users\user\AppData\Local\Temp/ipykernel_5796/1826565885.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["filterItem2"] = dfc.apply(fmcViewFilterItem, axis=1)
